In [1]:
#!wget --header="Host: downloader-default2h.disk.yandex.net" --header="User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-US,en;q=0.9,ru;q=0.8" --header="Referer: https://yadi.sk/d/2KZaTHeWIXq79w" "https://downloader-default2h.disk.yandex.net/rzip/9662d8963f53a3f162902b06d872a768de0761f1d57bcca29ed85ae1a19f90f6/5f8458de/QWpFSFdsWW5EMjY3Y3U5QnFIVDV4MkZJTytkR1YxOUp5aWtZbUNMR2EwOWFxeW1reGhBdlcxQ0V5d3lmVm1tUXEvSjZicG1SeU9Kb25UM1ZvWG5EYWc9PQ==?uid=0&filename=ForVadim.zip&disposition=attachment&hash=AjEHWlYnD267cu9BqHT5x2FIO%2BdGV19JyikYmCLGa09aqymkxhAvW1CEywyfVmmQq/J6bpmRyOJonT3VoXnDag%3D%3D&limit=0&owner_uid=204301488&tknv=v2&rtoken=lGxbL9ARbqbb&force_default=no&ycrid=na-32f56c4a6729a6576d0b2a22143f1e21-downloader17h" -c -O 'ForVadim.zip'

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

from torch import Tensor
import imutils
import pandas as pd
import numpy as np 
import argparse
import cv2
import os

In [3]:
class Inception(nn.Module):

    def __init__(self, in_channels, ch1x1, ch3x3red, pool_proj,
                 conv_block=None):
        super(Inception, self).__init__()
        if conv_block is None:
            conv_block = BasicConv2d
            
        self.branch1 = conv_block(in_channels, ch1x1, kernel_size=1)

        self.branch2 = conv_block(in_channels, ch3x3red, kernel_size=1)


    def _forward(self, x):
        branch1 = self.branch1(x)
        branch2 = self.branch2(x)


        outputs = [branch1, branch2]
        return outputs

    def forward(self, x):
        outputs = self._forward(x)
        return torch.cat(outputs, 1)

class BasicConv2d(nn.Module):

    def __init__(self, in_channels, out_channels, **kwargs):
        super(BasicConv2d, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, bias=False, **kwargs)

    def forward(self, x):
        x = self.conv(x)
        x = F.relu(x, inplace=False)
        return x

In [4]:
blocks = [BasicConv2d, Inception]
conv_block = blocks[0]
inception_block = blocks[1]

In [5]:
MODEL_PATH = "./weights/model_297.pt"
model = torch.load(MODEL_PATH)

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Device:', torch.cuda.get_device_name(torch.cuda.current_device()))

Device: GeForce RTX 2080 Ti


In [7]:
model.to(device)

Sequential(
  (0): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (classifier): Sequential(
    (0): LocalResponseNorm(5, alpha=9.9999997e-05, beta=0.75, k=1.0)
    (1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (2): Inception(
      (branch1): BasicConv2d(
        (conv): Conv2d(64, 8, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
      (branch2): BasicConv2d(
        (conv): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
    )
    (3): AdaptiveAvgPool2d(output_size=(22, 22))
    (4): Dropout(p=0.20000001,

In [ ]:
size = 224
preds = []

def predict_frame(currentFrame):
    currentFrame = np.array(currentFrame)
    currentFrame = cv2.resize(np.float32(currentFrame),(size,size))
    currentFrame = currentFrame.reshape(1,3,224,224).astype('float')
    currentFrame = currentFrame/255
    currentFrame = torch.from_numpy(currentFrame)
    currentFrame = currentFrame.to(device, dtype=torch.float)
    prob = loadedModel(currentFrame)
    #max_prob = max(prob[0])
    max_prob = prob.max(dim=1, keepdim=True)
    max_prob = 1 + max_prob[0]
    max_prob = max_prob.detach().cpu().numpy()
    pred = prob.argmax(dim=1, keepdim=True)
    
    pred = pred.cpu().numpy()
    
    if pred == [[2]]:
        return "car", max_prob
    elif pred == [[1]]:
        return "human", max_prob
    else:
        return "none", max_prob
    #if(max_prob>.90):
    #   return pred , max_prob
    

# main function
if __name__ == '__main__':

    loadedModel = model
    filename = "scene.avi"
    PATH = "./ForVadim/{}".format(filename)
    camera = cv2.VideoCapture(PATH)

    show_pred = False
    # loop until interrupted
    while (True):
        
        (grabbed,frame) = camera.read()
        frame = imutils.resize(frame,width = 700)
        #flip around y-axis
        frame = cv2.flip(frame,1)
        
        clone = frame.copy()
        
        (height,width) = frame.shape[:2]
        #potential speedup if converted to grayscale, net should intake grayscale as well
        #grayClone = cv2.cvtColor(clone,cv2.COLOR_BGR2GRAY)
        
        #frame = np.float32(frame)
        #frame = torch.from_numpy(frame)
        #frame = frame.to(device)
        label, prob = predict_frame(frame)

        keypress_toshow = cv2.waitKey(1)
        
        if(keypress_toshow == ord("e")):
            show_pred = not show_pred
        
        #hershey font (image, text, org, font, fontScale, color[, thickness[, lineType[, bottomLeftOrigin]]])
        if(show_pred):
            cv2.putText(clone , str(label)+' '+str(prob*100)+'%' , (30,30) , cv2.FONT_HERSHEY_DUPLEX , 1 , (0,255,0) , 1)

        #cv2.imshow("GrayClone",grayClone)
        
        #cv2.imshow("Frame", frame)

        cv2.imshow("Video Feed", clone)

        keypress = cv2.waitKey(1) & 0xFF

        if(keypress == ord("q")):
            break

camera.release()

cv2.destroyAllWindows()
